# Fruit Classifier with pretrained VGG with Transfer Learning
Name : Muhammad Husain Toding Bunga
Student Id : 1301153626

1. Setting up and uploading data files with pydrive to current session



In [11]:
!pip install pydrive

    100% |████████████████████████████████| 993kB 20.3MB/s 
  Running setup.py bdist_wheel for pydrive ... - \ done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built pydrive


In [0]:
# login untuk download file (gunakan akun sso)

import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1GMtY0UEPL0GTRMv7c_YirYD_pD_3IZuT'})
download.GetContentFile('rgb.zip')

In [14]:
!unzip rgb.zip

Archive:  rgb.zip
   creating: rgb/
   creating: rgb/apple_1/
  inflating: rgb/apple_1/apple_1_1_100_crop.png  
  inflating: rgb/apple_1/apple_1_1_101_crop.png  
  inflating: rgb/apple_1/apple_1_1_102_crop.png  
  inflating: rgb/apple_1/apple_1_1_103_crop.png  
  inflating: rgb/apple_1/apple_1_1_104_crop.png  
  inflating: rgb/apple_1/apple_1_1_105_crop.png  
  inflating: rgb/apple_1/apple_1_1_106_crop.png  
  inflating: rgb/apple_1/apple_1_1_107_crop.png  
  inflating: rgb/apple_1/apple_1_1_108_crop.png  
  inflating: rgb/apple_1/apple_1_1_109_crop.png  
  inflating: rgb/apple_1/apple_1_1_10_crop.png  
  inflating: rgb/apple_1/apple_1_1_110_crop.png  
  inflating: rgb/apple_1/apple_1_1_111_crop.png  
  inflating: rgb/apple_1/apple_1_1_112_crop.png  
  inflating: rgb/apple_1/apple_1_1_113_crop.png  
  inflating: rgb/apple_1/apple_1_1_114_crop.png  
  inflating: rgb/apple_1/apple_1_1_115_crop.png  
  inflating: rgb/apple_1/apple_1_1_116_crop.png  
  inflating: rgb/apple_1/apple_1_1_117_

2. Initializing data directory, image input size, number of classes and etc

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dropout, BatchNormalization, GlobalAveragePooling2D, GlobalMaxPooling2D

In [0]:
img_width, img_height = 224, 224
batch_size = 8
nb_epochs = 10
input_shape = (img_width, img_height, 3)
trainDirectory = './rgb'

num_classes = 32
nb_train_samples = 17045
nb_validation_samples = 4242  

## b. Augmenting and Splitting training data
Images will be augmented with shear, zoom, and horizontal
This way we will have more data to be trained

Images will also be splitted into validation and training, with ratio 2 : 8

In [17]:
# augmentasi data, sekaligus split hasilnya ke 0.2 validasi 0.8 training
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    trainDirectory,
    shuffle = False,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    trainDirectory, # same directory as training data
    target_size=(img_width, img_height),
    shuffle = False,
    batch_size=batch_size,
    class_mode='categorical',
    subset ='validation') # set as validation data

Found 17045 images belonging to 32 classes.
Found 4242 images belonging to 32 classes.


## c.Using pretrained model and adding fully connected layer

In [18]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import Model
from tensorflow.keras.layers import Input

inputs = Input(shape=input_shape)
vgg16 = VGG16(weights='imagenet', include_top=False)
x = vgg16(inputs)
x = Flatten()(x)
x =  Dense(1024, activation='relu')(x)
x =  Dropout(0.5)(x)
outputs =  Dense(num_classes, activation='softmax')(x)

pretrain_vgg = Model(inputs,outputs)

58892288/58889256 [==============================] - 6s 0us/step


In [0]:
for layer in vgg16.layers:
    layer.trainable = False
    
pretrain_vgg.compile(loss='categorical_crossentropy',
              optimizer= tf.keras.optimizers.Adam(lr=1e-4),
              metrics=['accuracy'])

In [20]:
# cek gpu google colab , aktifkan lewat Edit -> Notebook Settings -> Hardware Accelerator -> GPU
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [21]:
with tf.device(device_name):
  pretrain_vgg.fit_generator(
      train_generator,
      steps_per_epoch = train_generator.samples // batch_size,
      validation_data = validation_generator, 
      validation_steps = validation_generator.samples // batch_size,
      epochs = nb_epochs)

Epoch 1/10
2130/2130 [==============================] - 391s 183ms/step - loss: 2.5826 - acc: 0.2416 - val_loss: 1.8374 - val_acc: 0.4165
Epoch 2/10
2130/2130 [==============================] - 386s 181ms/step - loss: 1.5172 - acc: 0.4895 - val_loss: 1.3104 - val_acc: 0.5028
Epoch 3/10
2130/2130 [==============================] - 385s 181ms/step - loss: 1.1016 - acc: 0.6064 - val_loss: 1.0438 - val_acc: 0.6071
Epoch 4/10
2130/2130 [==============================] - 385s 181ms/step - loss: 0.8717 - acc: 0.6825 - val_loss: 0.9506 - val_acc: 0.6557
Epoch 5/10
2130/2130 [==============================] - 385s 181ms/step - loss: 0.7184 - acc: 0.7367 - val_loss: 0.9047 - val_acc: 0.6899
Epoch 6/10
2130/2130 [==============================] - 383s 180ms/step - loss: 0.6068 - acc: 0.7792 - val_loss: 0.7578 - val_acc: 0.7137
Epoch 7/10
2130/2130 [==============================] - 385s 181ms/step - loss: 0.5202 - acc: 0.8114 - val_loss: 0.7687 - val_acc: 0.7274
Epoch 8/10
2130/2130 [============

In [22]:
with tf.device(device_name):
  pretrain_vgg.fit_generator(
      train_generator,
      steps_per_epoch = train_generator.samples // batch_size,
      validation_data = validation_generator, 
      validation_steps = validation_generator.samples // batch_size,
      epochs = nb_epochs)

Epoch 1/10
2130/2130 [==============================] - 386s 181ms/step - loss: 0.3458 - acc: 0.8743 - val_loss: 0.6886 - val_acc: 0.7594
Epoch 2/10
2130/2130 [==============================] - 387s 182ms/step - loss: 0.3223 - acc: 0.8857 - val_loss: 0.6704 - val_acc: 0.7710
Epoch 3/10
2130/2130 [==============================] - 387s 182ms/step - loss: 0.2952 - acc: 0.8914 - val_loss: 0.6648 - val_acc: 0.7823
Epoch 4/10
2130/2130 [==============================] - 387s 182ms/step - loss: 0.2686 - acc: 0.9043 - val_loss: 0.6650 - val_acc: 0.7790
Epoch 5/10
2130/2130 [==============================] - 387s 182ms/step - loss: 0.2519 - acc: 0.9097 - val_loss: 0.6391 - val_acc: 0.7981
Epoch 6/10
2130/2130 [==============================] - 387s 182ms/step - loss: 0.2359 - acc: 0.9146 - val_loss: 0.6257 - val_acc: 0.8007
Epoch 7/10
2130/2130 [==============================] - 386s 181ms/step - loss: 0.2237 - acc: 0.9202 - val_loss: 0.6438 - val_acc: 0.8033
Epoch 8/10
2130/2130 [============

In [0]:
with tf.device(device_name):
  pretrain_vgg.fit_generator(
      train_generator,
      steps_per_epoch = train_generator.samples // batch_size,
      validation_data = validation_generator, 
      validation_steps = validation_generator.samples // batch_size,
      epochs = nb_epochs)

Epoch 1/10
2130/2130 [==============================] - 384s 180ms/step - loss: 0.1835 - acc: 0.9336 - val_loss: 0.6759 - val_acc: 0.7979
Epoch 2/10
2130/2130 [==============================] - 384s 180ms/step - loss: 0.1714 - acc: 0.9374 - val_loss: 0.7034 - val_acc: 0.7991
Epoch 3/10
2130/2130 [==============================] - 383s 180ms/step - loss: 0.1593 - acc: 0.9430 - val_loss: 0.6163 - val_acc: 0.8337
Epoch 4/10
2130/2130 [==============================] - 384s 181ms/step - loss: 0.1587 - acc: 0.9428 - val_loss: 0.6966 - val_acc: 0.8151
Epoch 5/10
2130/2130 [==============================] - 384s 180ms/step - loss: 0.1504 - acc: 0.9467 - val_loss: 0.6646 - val_acc: 0.8163
Epoch 6/10
2130/2130 [==============================] - 383s 180ms/step - loss: 0.1474 - acc: 0.9462 - val_loss: 0.7182 - val_acc: 0.8092
Epoch 7/10
2130/2130 [==============================] - 382s 179ms/step - loss: 0.1399 - acc: 0.9491 - val_loss: 0.7258 - val_acc: 0.8292
Epoch 8/10
2130/2130 [============